In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.font_manager as fm
import matplotlib
import timeit
import feather
%matplotlib inline

In [2]:
res = pd.read_feather('DBus_data_first_clean_ian.feather')

In [3]:
res.shape

(70904023, 11)

In [4]:
# Attaching leading zeroes to VehicleJourneyIDs and StopIDs of less than 4 digits
# code from: https://stackoverflow.com/questions/33243763/pandas-add-leading-0-to-string-values-so-all-values-are-equal-len

res.VehicleJourneyID = res.VehicleJourneyID.apply(lambda x: str(int(x)).zfill(4))
res.StopID = res.StopID.apply(lambda x: str(int(x)).zfill(4))

In [9]:
res.head(4)

,Timestamp,JourneyPatternID,TimeFrame,VehicleJourneyID,Lon,Lat,Delay,VehicleID,StopID,AtStop,Day
0,2013-01-31 00:00:01,00131005,2013-01-30,3406,-6.273923,53.343307,-235,33608,1998,False,Wednesday
1,2013-01-31 00:00:03,00071003,2013-01-30,6069,-6.230700,53.317432,0,43003,0008,True,Wednesday
2,2013-01-31 00:00:03,00671001,2013-01-30,0905,-6.325533,53.346302,-396,33438,2248,False,Wednesday
3,2013-01-31 00:00:03,00831001,2013-01-30,5613,-6.284449,53.420197,-490,40012,1552,False,Wednesday


### Part I. Second stage cleaning

#### 1. Delete those rows of same TimeFrame + JourneyPatternID  +  VehicleJourneyID which unique StopID under n.
* 0711: Delete those rows of same TimeFrame + JourneyPatternID + VehicleJourneyID + VehicleID occurance under n times. test n = 10
* 0806 revised:TimeFrame + JourneyPatternID  +  VehicleJourneyID which unique StopID under n. n = 5

In [24]:
gb = res.groupby(['TimeFrame', 'JourneyPatternID', 'VehicleJourneyID'])

In [26]:
# Delete those count is less than 10

gbcount = gb.StopID.nunique()
idx = pd.DataFrame(gbcount[gbcount < 5])

StopID
TimeFrame  JourneyPatternID VehicleJourneyID        
2012-11-05 00040001         5060                   1
                            5331                   1
           00041002         5156                   1
                            5416                   1
           00070001         6520                   1
                            6523                   1
                            6966                   3
           00071001         6855                   1
                            6991                   1
           00091001         4464                   1
           00110001         5281                   1
           00111002         5241                   1
           00130008         4094                   1
           00131012         4106                   1
           00141001         6263                   1
                            6275                   1
                            6315                   1
           00150001         5826                   1
           00151001         5582                   1
                            5929                   1
           00160001         5591                   1
                            5607                   4
           00161001         5652                   1
                            5656                   1
           00171001         0769                   1
           00180001         0878                   2
           00181001         0844                   4
           00250001         0713                   1
           00251001         0697                   1
           00270001         4760                   1
...                                              ...
2013-01-31 077A1001         3873                   1
                            3888                   1
                            3898                   1
                            4109                   1
                            982744                 1
           079A0001         2897                   1
                            2898                   1
                            2902                   1
                            2911                   1
                            2919                   1
                            2924                   1
           079A1001         2878                   1
                            2884                   1
                            2889                   1
                            2902                   1
                            2905                   1
                            2909                   1
                            2913                   1
           083A0001         5641                   1
           084A0001         5087                   1
                            5118                   1
           084A0002         4711                   1
           084A1001         5086                   1
                            5101                   1
           084A1002         4710                   1
           084X1001         5785                   1
                            5801                   1
           084X1002         5792                   1
                            5795                   1
                            5808                   1

[260467 rows x 1 columns]

In [27]:
# Set the index back to data

idx.reset_index(['TimeFrame', 'JourneyPatternID', 'VehicleJourneyID'], inplace=True)

In [33]:
# Merge back to res to see which one should be keep

merge = pd.merge(res, idx, on=['TimeFrame', 'JourneyPatternID', 'VehicleJourneyID'], how='outer',suffixes='oa')

# If null means that row should keep

res = merge[merge['StopIDa'].isnull()]

# Clean up

res.drop('StopIDa', axis=1, inplace=True)
res = res.rename(columns = {'StopIDo':'StopID'})
del merge

/Users/EmmaChang/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


#### 2. Delete those rows of same TimeFrame + VehicleJourneyID  but exist over one JourneyPatternID

In [35]:
# Due to make following function work need to change datatype here

res["JourneyPatternID"] = res["JourneyPatternID"].astype("object")

# Check how many unique JourneyPatternID under that group (if over 1 is abnormal)

gb = res.groupby(['TimeFrame', 'VehicleJourneyID'])
idx = pd.DataFrame(gb['JourneyPatternID'].unique())
idx['JourneyPatternID'] = idx['JourneyPatternID'].apply(lambda x: len(x))

In [36]:
# Get those rows which unique JourneyPatternID are over 1

idx = idx[idx.JourneyPatternID > 1]

# Set the index back to data

idx.reset_index(['TimeFrame', 'VehicleJourneyID'], inplace=True)


In [37]:
# Merge with res then can see if is null at the column JourneyPatternIDA then should keep
# suffixes='OA' : is set column name to JourneyPatternIDO and JourneyPatternIDA

merge = pd.merge(res, idx, on=['TimeFrame', 'VehicleJourneyID'], how='outer',suffixes='OA')

# Keep those JourneyPatternIDA is null

res = merge[merge['JourneyPatternIDA'].isnull()]

# Clean up

res.drop('JourneyPatternIDA', axis=1, inplace=True)
res = res.rename(columns = {'JourneyPatternIDO':'JourneyPatternID'})
del merge

In [38]:
# Store as feather

res = res.reset_index()
res.to_feather('DBus_stage_three_clean_v1_step1.feather')

In [40]:
res.shape

(65735408, 12)

In [ ]:
#res = pd.read_feather('DBus_stage_three_clean_v1_step1.feather')

0805 revised: 
First, find the possible stop location by calaulate maximum occurance AtStop=1 from gps data.
Second, use stopID as key to calculate distance with googletransit data.
=> Find three situation: 
(1) Google transit data exists duplicate stopID with different lon/lat.
(2) There are ? number of stopID exist in gps data but not in google transit data.
(3) There are 60 number of stopID location is over 200 meters difference from google transit data.

#### Prepare google transit stop data
Due to there are duplicate stop short name in google transit data. We compare the location to gps data, we found out other than stop_id has 'VIR' string in there should delete. 8250DB00661, 8250DB002561, 8220PB003854, 8220DB007270 also should delete. (The same process should apply in database)

In [41]:
# First, clean google transit stop location data.
stops_2012 = pd.read_csv('stops.txt')
stops_2012.head(5)

,stop_id,stop_name,stop_lat,stop_lon
0,8220DB000002,"Parnell Square, Rotunda",53.352244,-6.263693
1,8220DB000003,"Parnell Square, Rotunda",53.352308,-6.263781
2,8220DB000004,"Parnell Square, Rotunda",53.352565,-6.264161
3,8220DB000006,"Parnell Square, Rotunda",53.352740,-6.264439
4,8220DB000007,"Parnell Square, Rotunda",53.352832,-6.264556


In [42]:
stops_2012 = stops_2012[stops_2012.stop_id.str.find('VIR') == -1]
stops_2012['StopID'] = stops_2012['stop_id'].str[-4:]
# Revised: 
# stops_2012 = stops_2012[~stops_2012.stop_id.isin(['8250DB00661', '8250DB002561', '8220PB003854', '8220DB007270'])]
stops_2012.set_value(index = stops_2012[(stops_2012.stop_id == '8250DB00661')].index, col='StopID', value='0661_1')
stops_2012.set_value(index = stops_2012[(stops_2012.stop_id == '8250DB002561')].index, col='StopID', value='2561_1')
stops_2012.set_value(index = stops_2012[(stops_2012.stop_id == '8220PB003854')].index, col='StopID', value='3854_1')
stops_2012.set_value(index = stops_2012[(stops_2012.stop_id == '8220DB007270')].index, col='StopID', value='7270_1')

,stop_id,stop_name,stop_lat,stop_lon,StopID
0,8220DB000002,"Parnell Square, Rotunda",53.352244,-6.263693,0002
1,8220DB000003,"Parnell Square, Rotunda",53.352308,-6.263781,0003
2,8220DB000004,"Parnell Square, Rotunda",53.352565,-6.264161,0004
3,8220DB000006,"Parnell Square, Rotunda",53.352740,-6.264439,0006
4,8220DB000007,"Parnell Square, Rotunda",53.352832,-6.264556,0007
5,8220DB000008,"Parnell Square, Rotunda",53.353263,-6.265169,0008
6,8220DB000010,"Parnell Square, Rotunda",53.353383,-6.265389,0010
7,8220DB000011,"Dorset St, Phibsborough",53.357098,-6.264367,0011
8,8220DB000012,"Dorset St, Dublin",53.356787,-6.264620,0012
9,8220DB000014,"Dorset St, Phibsborough",53.358540,-6.262731,0014


In [43]:
# Make sure no duplicates.
stops_2012[stops_2012['StopID'].duplicated()]

,stop_id,stop_name,stop_lat,stop_lon,StopID


### Part II. Replace StopID that not exist in googletransit data to stop_id that is close to it.
Origin: Delete stopID not in that trip_id (same meaning as JourneyPatternID) Revised:0805

#### Step 1. Find possible position of StopID in gps data

In [44]:
res_temp = res
res_temp["JourneyPatternID"] = res_temp["JourneyPatternID"].astype("category")
res_temp = res_temp[res_temp.AtStop == True]

* Longtitude

In [45]:
# First groupby StopID with Lon and found the max count

gb = res_temp.groupby(['StopID','Lon'])
gbLon = pd.DataFrame(gb['AtStop'].count())
gbLon.reset_index(['StopID','Lon'], inplace=True)

In [46]:
# Find the maximun occurance Lon for each StopID

maxCountLon = gbLon[gbLon.groupby(['StopID'])['AtStop'].transform('max') == gbLon['AtStop']]
maxCountLon.shape

(4733, 3)

* Latitude

In [47]:
# First groupby StopID with Lon and found the max count

gb = res_temp.groupby(['StopID','Lat'])
gbLat = pd.DataFrame(gb['AtStop'].count())
gbLat.reset_index(['StopID','Lat'], inplace=True)

In [48]:
# Find the maximun occurance Lat for each StopID

maxCountLat = gbLat[gbLat.groupby(['StopID'])['AtStop'].transform('max') == gbLat['AtStop']]
maxCountLat.shape

(4724, 3)

* Lon/Lat

In [49]:
# Merge by StopID

maxCountLonLat = pd.merge(maxCountLon, maxCountLat, on=['StopID'], how='inner')
maxCountLonLat.shape

(4855, 5)

In [50]:
# There are 8 StopID cannot find the obvious Co-ordinate, use average

gb = maxCountLonLat.groupby('StopID')
gb = gb.mean()
gb.reset_index(['StopID'], inplace=True)

# Don't know why there are some StopID not exist appear in the index

gb.dropna(axis=0, how='any', inplace=True)
gb.drop(['AtStop_x', 'AtStop_y'], axis=1, inplace=True)
gps_est_lat_lon = gb.loc[:, ['StopID', 'Lat', 'Lon']]

In [51]:
# Now we have the possible lat/lon of each StopID
gps_est_lat_lon.head(3)

,StopID,Lat,Lon
0,0010,53.353436,-6.265345
1,0100,53.303043,-6.322296
2,1000,53.390884,-6.201829


#### Step 2. For those StopID exist in gps data not in googletransit data find the replacement StopID

In [52]:
# Merge google_transit data with possible lat/lon of gps data
compare_google12_gpsest = pd.merge(gps_est_lat_lon, stops_2012, on='StopID', how='outer')
compare_google12_gpsest.head(2)

,StopID,Lat,Lon,stop_id,stop_name,stop_lat,stop_lon
0,0010,53.353436,-6.265345,8220DB000010,"Parnell Square, Rotunda",53.353383,-6.265389
1,0100,53.303043,-6.322296,8230DB000100,"Wellington Lane, Templeogue",53.303083,-6.322361


In [53]:
# Find StopID exist in gps data not in google transit data
not_in_google12 = compare_google12_gpsest[compare_google12_gpsest.stop_name.isnull()]
not_in_google12.drop(['stop_id', 'stop_name', 'stop_lat', 'stop_lon'], axis=1, inplace=True)
not_in_google12.head(3)

/Users/EmmaChang/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,StopID,Lat,Lon
2333,3581,53.446697,-6.171781
2334,3582,53.448593,-6.167941
2388,3637,53.448624,-6.167745


In [54]:
# Production
stops_2012_temp = stops_2012
stops_2012_temp['merge_id'] = np.ones((stops_2012_temp.shape[0],1 ), dtype=np.int16)
not_in_google12['merge_id'] = np.ones((not_in_google12.shape[0],1 ), dtype=np.int16)
production = pd.merge(stops_2012_temp, not_in_google12, on='merge_id', suffixes=('_google','_gps'))
production.head(3)

/Users/EmmaChang/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,stop_id,stop_name,stop_lat,stop_lon,StopID_google,merge_id,StopID_gps,Lat,Lon
0,8220DB000002,"Parnell Square, Rotunda",53.352244,-6.263693,0002,1,3581,53.446697,-6.171781
1,8220DB000002,"Parnell Square, Rotunda",53.352244,-6.263693,0002,1,3582,53.448593,-6.167941
2,8220DB000002,"Parnell Square, Rotunda",53.352244,-6.263693,0002,1,3637,53.448624,-6.167745


In [55]:
# Calculate distance
from geopy.distance import great_circle
production['distance_diff'] = production.loc[:, ['Lat','Lon','stop_lat','stop_lon']].apply(lambda x: great_circle((x[0],x[1]), (x[2],x[3])).meters, axis=1)
production.head(2)

,stop_id,stop_name,stop_lat,stop_lon,StopID_google,merge_id,StopID_gps,Lat,Lon,distance_diff
0,8220DB000002,"Parnell Square, Rotunda",53.352244,-6.263693,0002,1,3581,53.446697,-6.171781,12145.871698
1,8220DB000002,"Parnell Square, Rotunda",53.352244,-6.263693,0002,1,3582,53.448593,-6.167941,12456.518848


In [56]:
# Find the minmum as 
production_min = production[production.groupby(['StopID_gps'])['distance_diff'].transform('min') == production['distance_diff']]
production_min.head(2)

,stop_id,stop_name,stop_lat,stop_lon,StopID_google,merge_id,StopID_gps,Lat,Lon,distance_diff
16044,8220DB000339,"Burgh Quay, Dublin City South",53.347255,-6.256896,0339,1,7543,53.347473,-6.255754,79.634691
16046,8220DB000339,"Burgh Quay, Dublin City South",53.347255,-6.256896,0339,1,7545,53.347420,-6.256560,28.881757


#### Observation
From table can find out that, duplicate in googletransit data:
* 8220DB007270 -> is 7220 in gps data
* 8220PB003854,  8250DB00661 can drop
* 8250DB002561 -> is 7550 in gps

In [57]:
# So we will replace 8220DB007270 to '7220' and 8250DB002561 to '7250' in google transit.
# Others replace gps StopID to corresponding StopID_google
# Here drop StopID_google == '3854_1' is because duplicate
production_min = production_min[~production_min.StopID_google.isin(['3854_1', '7270_1', '2561_1'])]

#### Solution
Replace those distance < 150 to google transit stop_id

In [58]:
replace = production_min[production_min.distance_diff < 150].loc[:,['StopID_gps', 'StopID_google']]
delete = production_min[production_min.distance_diff >= 150].loc[:,['StopID_gps', 'StopID_google']]

In [59]:
delete.head(3)

,StopID_gps,StopID_google
163078,7482,0323
163152,7502,0324
163199,7479,0327


#### Step 3. Replace and delete

In [60]:
temp = pd.merge(res, delete, left_on='StopID', right_on='StopID_gps', how='outer')
temp = temp[temp.StopID_gps.isnull()]


In [61]:
temp.drop(['index', 'StopID_gps', 'StopID_google'], axis=1, inplace=True)

In [62]:
temp2 = pd.merge(temp, replace, left_on='StopID', right_on='StopID_gps', how='outer')
temp2['StopID'] = temp2.loc[:,['StopID_google', 'StopID']].apply(lambda x: x[1] if pd.isnull(x[0]) else x[0], axis=1)

In [63]:
res = temp2.loc[:,['Timestamp', 'JourneyPatternID', 'TimeFrame', 'VehicleJourneyID', 'Lon', 'Lat', 'VehicleID', 'StopID', 'AtStop', 'Day']]

In [64]:
res = res.reset_index()
res.to_feather('DBus_stage_three_clean_v1_step2.feather')

### Part III. Add Weather Information

In [65]:
res = pd.read_feather('DBus_stage_three_clean_v1_step2.feather')
res

,index,Timestamp,JourneyPatternID,TimeFrame,VehicleJourneyID,Lon,Lat,VehicleID,StopID,AtStop,Day
0,0,2013-01-31 00:00:01,00131005,2013-01-30,3406,-6.273923,53.343307,33608,1998,False,Wednesday
1,1,2013-01-30 23:59:04,00131005,2013-01-30,3406,-6.278856,53.342804,33608,1998,False,Wednesday
2,2,2013-01-30 23:59:22,00131005,2013-01-30,3406,-6.277600,53.342831,33608,1998,False,Wednesday
3,3,2013-01-30 23:59:43,00131005,2013-01-30,3406,-6.276123,53.343075,33608,1998,False,Wednesday
4,4,2013-01-30 23:07:25,00131002,2013-01-30,3341,-6.280505,53.343040,33343,1998,True,Wednesday
5,5,2013-01-30 23:07:47,00131002,2013-01-30,3341,-6.280505,53.343040,33343,1998,True,Wednesday
6,6,2013-01-30 23:08:05,00131002,2013-01-30,3341,-6.280198,53.342987,33343,1998,False,Wednesday
7,7,2013-01-30 23:08:26,00131002,2013-01-30,3341,-6.279657,53.342892,33343,1998,False,Wednesday
8,8,2013-01-30 23:08:44,00131002,2013-01-30,3341,-6.279628,53.342888,33343,1998,False,Wednesday
9,9,2013-01-30 23:09:07,00131002,2013-01-30,3341,-6.278461,53.342800,33343,1998,False,Wednesday


In [66]:
# Read weather data
ch = pd.read_csv('Casement_Hourly_Nov12Jan13_WindRainOnly.csv')
dh = pd.read_csv('DubAirport_Hourly_Nov12Jan13_WindRainOnly.csv')
ph = pd.read_csv('PhoenixPark_Hourly_Nov12Jan13_RainOnly.csv')

In [67]:
dh.dtypes

date     object
rain    float64
wdsp      int64
dtype: object

In [68]:
from datetime import datetime
ch['DateTime'] = ch['date'].apply(lambda x: datetime.strptime(x, '%m/%d/%Y %H:%M'))
ch['DateTime'] = ch['DateTime'].apply(lambda x: datetime.strftime(x, '%Y-%m-%d %H'))

dh['DateTime'] = dh['date'].apply(lambda x: datetime.strptime(x, '%m/%d/%Y %H:%M'))
dh['DateTime'] = dh['DateTime'].apply(lambda x: datetime.strftime(x, '%Y-%m-%d %H'))


ph['DateTime'] = ph['date'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M'))
ph['DateTime'] = ph['DateTime'].apply(lambda x: datetime.strftime(x, '%Y-%m-%d %H'))

In [69]:
ch.drop('date', axis=1, inplace=True)
dh.drop('date', axis=1, inplace=True)
ph.drop('date', axis=1, inplace=True)

In [70]:
# Merge
merge = pd.merge(ch, dh, on='DateTime', how='outer')
merge = pd.merge(merge, ph, on='DateTime', how='outer')
merge.head(5)

,rain_x,wdsp_x,DateTime,rain_y,wdsp_y,rain
0,0.0,8,2012-11-06 00,0.0,10,0
1,0.0,8,2012-11-06 01,0.0,10,0
2,0.0,10,2012-11-06 02,0.0,10,0
3,0.0,11,2012-11-06 03,0.0,13,0
4,0.0,10,2012-11-06 04,0.0,12,0


In [71]:
merge.isnull().sum()

rain_x      0
wdsp_x      0
DateTime    0
rain_y      0
wdsp_y      0
rain        0
dtype: int64

In [72]:
# Convert to float
merge['rain'] = merge['rain'].convert_objects(convert_numeric=True)
merge.dtypes

/Users/EmmaChang/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  


rain_x      float64
wdsp_x        int64
DateTime     object
rain_y      float64
wdsp_y        int64
rain        float64
dtype: object

In [73]:
# Calculate average
merge['Rain_Avg'] = merge.loc[:,['rain_x', 'rain_y', 'rain']].mean(axis=1)
merge['Wind_Speed_Avg'] = merge.loc[:,['wdsp_x', 'wdsp_y']].mean(axis=1)
weather = merge.loc[:,['DateTime', 'Rain_Avg', 'Wind_Speed_Avg']]
weather

,DateTime,Rain_Avg,Wind_Speed_Avg
0,2012-11-06 00,0.000000,9.0
1,2012-11-06 01,0.000000,9.0
2,2012-11-06 02,0.000000,10.0
3,2012-11-06 03,0.000000,12.0
4,2012-11-06 04,0.000000,11.0
5,2012-11-06 05,0.000000,11.5
6,2012-11-06 06,0.000000,12.0
7,2012-11-06 07,0.033333,14.5
8,2012-11-06 08,0.000000,15.0
9,2012-11-06 09,0.000000,15.5


In [74]:
# Merge with gps data
res['DateTime'] = res['Timestamp'].apply(lambda x: datetime.strftime(x, '%Y-%m-%d %H'))
res_2 = pd.merge(res, weather, on='DateTime', how='left')
res_2.head(5)

,index,Timestamp,JourneyPatternID,TimeFrame,VehicleJourneyID,Lon,Lat,VehicleID,StopID,AtStop,Day,DateTime,Rain_Avg,Wind_Speed_Avg
0,0,2013-01-31 00:00:01,00131005,2013-01-30,3406,-6.273923,53.343307,33608,1998,False,Wednesday,2013-01-31 00,0.0,19.5
1,1,2013-01-30 23:59:04,00131005,2013-01-30,3406,-6.278856,53.342804,33608,1998,False,Wednesday,2013-01-30 23,0.0,15.5
2,2,2013-01-30 23:59:22,00131005,2013-01-30,3406,-6.277600,53.342831,33608,1998,False,Wednesday,2013-01-30 23,0.0,15.5
3,3,2013-01-30 23:59:43,00131005,2013-01-30,3406,-6.276123,53.343075,33608,1998,False,Wednesday,2013-01-30 23,0.0,15.5
4,4,2013-01-30 23:07:25,00131002,2013-01-30,3341,-6.280505,53.343040,33343,1998,True,Wednesday,2013-01-30 23,0.0,15.5


In [75]:
del res

### Part III. Add School Holiday

In [76]:
school_holiday = ['2013-01-01','2013-01-02', '2013-01-03','2013-01-04','2013-01-05','2013-01-06']
school_holiday_days = ['Saturday','Sunday']
res_2['SchoolHoliday'] = np.zeros(res_2.shape[0], dtype=bool)

In [77]:
res_2.loc[res_2.TimeFrame.isin(school_holiday), 'SchoolHoliday'] = True
res_2.loc[res_2.Day.isin(school_holiday_days), 'SchoolHoliday'] = True

In [78]:
res_2.drop('DateTime', axis=1, inplace=True)

In [79]:
res_2.to_feather('DBus_stage_two_clean_v4.feather')

In [82]:
res_2.shape

(65640029, 14)

### Part IV. Add Feature: Number of Stops On Route (JPID_length)

In [ ]:
res = pd.read_feather('DBus_stage_two_clean_v4.feather')
res.drop('index', axis=1, inplace=True)
res.head(3)

In [84]:
res = res_2
del res_2

In [85]:
# Groupby TimeFrame + JourneyPatternID + VehicleJourneyID, count the number of unique stops on the journey
gb = res.groupby(['TimeFrame', 'JourneyPatternID', 'VehicleJourneyID'])
gbcount = gb.StopID.nunique()

In [88]:
idx = gbcount.reset_index(['TimeFrame', 'JourneyPatternID', 'VehicleJourneyID'])
idx.rename(columns = {'StopID':'JPID_length'}, inplace=True)

In [91]:
# Merge with res
res = pd.merge(res, idx, on=['TimeFrame', 'JourneyPatternID', 'VehicleJourneyID'], how='inner')

,index,Timestamp,JourneyPatternID,TimeFrame,VehicleJourneyID,Lon,Lat,VehicleID,StopID,AtStop,Day,Rain_Avg,Wind_Speed_Avg,SchoolHoliday,CountStopID
0,0,2013-01-31 00:00:01,00131005,2013-01-30,3406,-6.273923,53.343307,33608,1998,False,Wednesday,0.0,19.5,False,48
1,1,2013-01-30 23:59:04,00131005,2013-01-30,3406,-6.278856,53.342804,33608,1998,False,Wednesday,0.0,15.5,False,48
2,2,2013-01-30 23:59:22,00131005,2013-01-30,3406,-6.277600,53.342831,33608,1998,False,Wednesday,0.0,15.5,False,48
3,3,2013-01-30 23:59:43,00131005,2013-01-30,3406,-6.276123,53.343075,33608,1998,False,Wednesday,0.0,15.5,False,48
4,83610,2013-01-31 00:00:23,00131005,2013-01-30,3406,-6.272169,53.342979,33608,2001,False,Wednesday,0.0,19.5,False,48
5,83611,2013-01-31 00:00:42,00131005,2013-01-30,3406,-6.271217,53.343029,33608,2001,False,Wednesday,0.0,19.5,False,48
6,83612,2013-01-31 00:01:02,00131005,2013-01-30,3406,-6.270058,53.343559,33608,2001,False,Wednesday,0.0,19.5,False,48
7,148148,2013-01-31 00:01:21,00131005,2013-01-30,3406,-6.268173,53.343979,33608,2002,False,Wednesday,0.0,19.5,False,48
8,148149,2013-01-31 00:01:41,00131005,2013-01-30,3406,-6.265165,53.344200,33608,2002,False,Wednesday,0.0,19.5,False,48
9,148150,2013-01-31 00:02:02,00131005,2013-01-30,3406,-6.264954,53.344200,33608,2002,False,Wednesday,0.0,19.5,False,48


### Add Feature : Is XBuses or not

In [7]:
res['XBuses'] = res['JourneyPatternID'].apply(lambda x: '1' if x.find('X') != -1 else '0')

In [5]:
res.rename(columns = {'CountStopID':'JPID_length'}, inplace=True)

In [8]:
res.head(2)

,index,Timestamp,JourneyPatternID,TimeFrame,VehicleJourneyID,Lon,Lat,VehicleID,StopID,AtStop,Day,Rain_Avg,Wind_Speed_Avg,SchoolHoliday,JPID_length,XBuses
0,0,2013-01-31 00:00:01,00131005,2013-01-30,3406,-6.273923,53.343307,33608,1998,False,Wednesday,0.0,19.5,False,48,0
1,1,2013-01-30 23:59:04,00131005,2013-01-30,3406,-6.278856,53.342804,33608,1998,False,Wednesday,0.0,15.5,False,48,0


In [9]:
# Output to feather
res.drop('index', axis=1, inplace=True)
res.reset_index(inplace=True)
res.to_feather('DBus_stage_two_clean_final.feather')

### Create an input to add the feature JPID_journeys

In [11]:
res.sort_values(['TimeFrame', 'JourneyPatternID', 'VehicleJourneyID', 'Timestamp'], inplace=True)

In [13]:
gb = res.groupby(['TimeFrame', 'JourneyPatternID', 'VehicleJourneyID', 'StopID'])

In [14]:
# Only keep first row
first = gb.Timestamp.first()

In [17]:
first = first.reset_index(['TimeFrame', 'JourneyPatternID', 'VehicleJourneyID', 'StopID'])
first.drop('Timestamp', axis=1, inplace=True)
first.head(4)

,TimeFrame,JourneyPatternID,VehicleJourneyID,StopID,Timestamp
0,2012-11-05,00070001,6963,2040,2012-11-06 00:00:17
1,2012-11-05,00070001,6963,2041,2012-11-06 00:00:55
2,2012-11-05,00070001,6963,2043,2012-11-06 00:02:16
3,2012-11-05,00070001,6963,2044,2012-11-06 00:02:36
4,2012-11-05,00070001,6963,2045,2012-11-06 00:02:57
5,2012-11-05,00070001,6963,2046,2012-11-06 00:03:37
6,2012-11-05,00070001,6963,3205,2012-11-06 00:05:37
7,2012-11-05,00070001,6963,3206,2012-11-06 00:05:57
8,2012-11-05,00070001,6963,3207,2012-11-06 00:06:15
9,2012-11-05,00070001,6963,3208,2012-11-06 00:06:35


In [28]:
# In order to get the SSID, move down the orginal dataframe one step.
first_2 = first.loc[1:]

# In order to concat move down dataframe to original one, need the number of rows the same as original one.
# So fill in one row to last row.
first_last = first.loc[16663374:]
first_move_down = pd.concat([first_2, first_last], axis=0)

,TimeFrame,JourneyPatternID,VehicleJourneyID,StopID
1,2012-11-05,00070001,6963,2041
2,2012-11-05,00070001,6963,2043
3,2012-11-05,00070001,6963,2044
4,2012-11-05,00070001,6963,2045
5,2012-11-05,00070001,6963,2046
6,2012-11-05,00070001,6963,3205
7,2012-11-05,00070001,6963,3206
8,2012-11-05,00070001,6963,3207
9,2012-11-05,00070001,6963,3208
10,2012-11-05,00070001,6963,3209


In [42]:
# Rename column
first_move_down.columns = pd.Series(['TimeFrame_m', 'JourneyPatternID_m', 'VehicleJourneyID_m', 'StopID_m'])
first_move_down.reset_index(inplace=True, drop=True)

In [44]:
# Now concat two dataframe together
get_ssid = pd.concat([first, first_move_down], axis=1)

In [46]:
get_ssid.head(3)

,TimeFrame,JourneyPatternID,VehicleJourneyID,StopID,TimeFrame_m,JourneyPatternID_m,VehicleJourneyID_m,StopID_m
0,2012-11-05,00070001,6963,2040,2012-11-05,00070001,6963,2041
1,2012-11-05,00070001,6963,2041,2012-11-05,00070001,6963,2043
2,2012-11-05,00070001,6963,2043,2012-11-05,00070001,6963,2044


In [47]:
# In order to make sure there are the same JPID, use boolean to check
get_ssid['SameJPID'] = np.where(get_ssid['JourneyPatternID'] == get_ssid['JourneyPatternID_m'], True, False)
get_ssid.head(3)

,TimeFrame,JourneyPatternID,VehicleJourneyID,StopID,TimeFrame_m,JourneyPatternID_m,VehicleJourneyID_m,StopID_m,SameJPID
0,2012-11-05,00070001,6963,2040,2012-11-05,00070001,6963,2041,True
1,2012-11-05,00070001,6963,2041,2012-11-05,00070001,6963,2043,True
2,2012-11-05,00070001,6963,2043,2012-11-05,00070001,6963,2044,True


In [49]:
# Get the SSID of each group
get_ssid['SSID'] = np.where(get_ssid['SameJPID'], get_ssid['StopID'] + get_ssid['StopID_m'], np.nan)
get_ssid

,TimeFrame,JourneyPatternID,VehicleJourneyID,StopID,TimeFrame_m,JourneyPatternID_m,VehicleJourneyID_m,StopID_m,SameJPID,SSID
0,2012-11-05,00070001,6963,2040,2012-11-05,00070001,6963,2041,True,20402041
1,2012-11-05,00070001,6963,2041,2012-11-05,00070001,6963,2043,True,20412043
2,2012-11-05,00070001,6963,2043,2012-11-05,00070001,6963,2044,True,20432044
3,2012-11-05,00070001,6963,2044,2012-11-05,00070001,6963,2045,True,20442045
4,2012-11-05,00070001,6963,2045,2012-11-05,00070001,6963,2046,True,20452046
5,2012-11-05,00070001,6963,2046,2012-11-05,00070001,6963,3205,True,20463205
6,2012-11-05,00070001,6963,3205,2012-11-05,00070001,6963,3206,True,32053206
7,2012-11-05,00070001,6963,3206,2012-11-05,00070001,6963,3207,True,32063207
8,2012-11-05,00070001,6963,3207,2012-11-05,00070001,6963,3208,True,32073208
9,2012-11-05,00070001,6963,3208,2012-11-05,00070001,6963,3209,True,32083209


In [50]:
# 
gb = get_ssid.groupby(['JourneyPatternID', 'SSID'])
count = gb.count()

In [53]:
# Get JPID to its SSID
JPID_SSID = count[count.StopID > 0]
JPID_SSID.reset_index(['JourneyPatternID', 'SSID'], inplace=True)
JPID_SSID = JPID_SSID.loc[:, ['JourneyPatternID', 'SSID', 'StopID']]
JPID_SSID

,JourneyPatternID,SSID,StopID
0,00010001,00440045,1758.0
1,00010001,00440046,20.0
2,00010001,00440048,2.0
3,00010001,00440119,2.0
4,00010001,00440213,1.0
5,00010001,00450046,1763.0
6,00010001,00450047,5.0
7,00010001,00450119,2.0
8,00010001,00450213,1.0
9,00010001,00460047,1723.0


In [55]:
# Get JPID_journeys of each SSID
gb = JPID_SSID.groupby('SSID')
gbcount = gb.JourneyPatternID.nunique()
gbcount = gbcount.reset_index('SSID')


SSID
00020057    4
00020058    3
00020059    2
00020192    5
00020274    6
00020320    2
00020461    1
00020783    1
00020784    2
00020785    1
00021813    1
00030012    1
00030014    1
00030053    1
00030054    1
00030055    1
00030056    3
00030058    2
00030064    1
00030071    1
00030072    1
00030192    1
00030310    1
00031348    1
00040027    1
00040028    1
00040029    1
00040090    1
00040091    1
00040129    1
           ..
75410296    1
75410299    1
75410614    1
75410615    1
75410617    1
75410664    1
75410905    1
75411212    1
75413605    1
75413606    1
75413639    1
75500340    1
75500350    1
75500351    1
75500352    1
75501069    1
75501070    1
75501071    1
75501072    1
75501155    1
75501358    2
75501424    1
75501436    1
75502002    1
75502183    1
75502184    1
75502346    1
75502357    2
75502425    1
75502590    1
Name: JourneyPatternID, Length: 36665, dtype: int64

In [62]:
#
gbcount.rename(columns = {'JourneyPatternID':'JPID_journeys'}, inplace=True)
gbcount.to_csv('input4_JPID_journeys.csv', index=False)